<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/Catalyst_Optimizer_Logical%26Physical_plans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=a5318b7fcd4e2e8595172956ab51c7fb6a6fd411d4a417289993a91f55f28569
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
import pyspark

Запустим код и увидим огромную простыню логов. Стоит отметить, что все планы запросов читаются сверху вниз.



In [5]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Catalyst Optimizer Example").getOrCreate()


data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])


result = df.filter(df.Age > 30).select("Name", "Age").orderBy("Name")

# Показываем логический и физический планы
result.explain(True)

== Parsed Logical Plan ==
'Sort ['Name ASC NULLS FIRST], true
+- Project [Name#0, Age#1L]
   +- Filter (Age#1L > cast(30 as bigint))
      +- LogicalRDD [Name#0, Age#1L], false

== Analyzed Logical Plan ==
Name: string, Age: bigint
Sort [Name#0 ASC NULLS FIRST], true
+- Project [Name#0, Age#1L]
   +- Filter (Age#1L > cast(30 as bigint))
      +- LogicalRDD [Name#0, Age#1L], false

== Optimized Logical Plan ==
Sort [Name#0 ASC NULLS FIRST], true
+- Filter (isnotnull(Age#1L) AND (Age#1L > 30))
   +- LogicalRDD [Name#0, Age#1L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Name#0 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Name#0 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=12]
      +- Filter (isnotnull(Age#1L) AND (Age#1L > 30))
         +- Scan ExistingRDD[Name#0,Age#1L]



Начнем с первого абзаца. **Parsed Logical Plan** показывает первосозданный, неоптимизированный план выполнения запроса. Важно, что *он пока что не смотрит на типы данных*.

- Sort ['Name ASC NULLS FIRST], true: Сортировка по столбцу Name по возрастанию с NULL значениями первыми.
- Project ['Name, 'Age]: Выбор столбцов Name и Age.
- Filter ('Age > 30): Фильтрация строк, где Age больше 30.

Двигаемся дальше. Далее у нас **Analyzed Logical Plan**. На этом этапе Catalyst анализирует и проверяет логический план, добавляя *информацию о типах данных и идентификаторах столбцов*. Заметим также, что *появились ссылки на столбцы*.

- Name: string, Age: int: Информация о типах данных для столбцов.
- Sort [Name#2 ASC NULLS FIRST], true: Сортировка по столбцу Name (идентификатор столбца Name#2).
- Project [Name#2, Age#3]: Выбор столбцов Name (идентификатор Name#2) и Age (идентификатор Age#3).
- Filter (Age#3 > 30): Фильтрация строк, где Age (идентификатор Age#3) больше 30.

 Двигаемся еще ниже. Это уже **Optimized Logical Plan**. Видим, что оптимизаций никаких не произошло, ведь запрос заранее составлен правильно.



И, наконец, посмотрим в конец простыни логов и увидим **Physical Plan**. Он достаточно прост.